<a href="https://colab.research.google.com/github/iBibek/dsci6015-lab1/blob/main/Lab_3_1_Phish_and_Spam_Classification_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##############---------------
##############---------------
##############---------------
##############--------------- Data Processing
##############---------------
##############---------------
##############---------------


In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
phishing_dataset = np.genfromtxt('/content/drive/MyDrive/6015/phishing_dataset.csv', delimiter=',', dtype=np.int32)
X = phishing_dataset[:,:-1]  # Samples
y = phishing_dataset[:, -1] #targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
sc = StandardScaler()
sc.fit(X_train)
# Apply the scaler to the X training data
X_train_std = sc.transform(X_train)
# Apply the SAME scaler to the X test data
X_test_std = sc.transform(X_test)

In [ ]:
##############---------------
##############---------------
##############---------------
##############--------------- Linear Regression Model 
##############---------------
##############---------------
##############---------------


In [69]:
import torch
class LinearRegression(torch.nn.Module):   
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(30, 1)    
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

model = LinearRegression()

criterion = torch.nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

X_train_std= torch.from_numpy(np.array(X_train_std).astype(np.float32))
y_train=torch.from_numpy(np.array(y_train).astype(np.float32))

for epoch in range(700):
    model.train()
    optimizer.zero_grad()    
    y_pred = model(X_train_std)    
     
    loss = criterion(y_pred.reshape(-1), y_train)    # Backward pass
    loss.backward()
    optimizer.step()

    

In [71]:

model.eval()
X_test_std=torch.from_numpy(np.array(X_test_std).astype(np.float32))
tar_pred = model(X_test_std)
tr=np.array(y_test) == 1
op=np.array(tar_pred.cpu().detach().numpy()) >=0.5
print("The accuracy for Linear Regression Model is: ", 100* accuracy_score(tr,op))

The accuracy for Linear Regression Model is:  80.97678625263792


LOGISTIC REGRESSION

In [39]:
##############---------------
##############---------------
##############---------------
##############---------------Logistic Regression Model 
##############---------------
##############---------------
##############---------------


In [23]:
import torch
from torch.nn import functional as F
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(30, 1)     
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred
modelR = LogisticRegression()

# we need to change the faetures to 0 and 1 only
#y_train=[ 0 if x<=0 else 1 for x in training_targets ]
X_train_std=torch.from_numpy(np.array(X_train_std).astype(np.float32))
y_train=torch.from_numpy(np.array(y_train).astype(np.float32))
# use the  Binary Cross Entropy for loss since we are predicting Binary Value as op
criterion = torch.nn.BCELoss(size_average=True)
optimizer=torch.optim.SGD(modelR.parameters(),lr=0.01)
for epoch in range(500):
    modelR.train()
    optimizer.zero_grad()    
    y_pred = modelR(X_train_std)    
    #use the binary cross entropy loss with the logits,
    loss=torch.nn.functional.binary_cross_entropy_with_logits(y_pred.reshape(-1), y_train)
    loss.backward()
    optimizer.step()


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [38]:
modelR.eval()
X_test_std=torch.from_numpy(np.array(X_test_std).astype(np.float32))
tar_pred = modelR(X_test_std)
tr=np.array(y_test) == 1
op=np.array(tar_pred.cpu().detach().numpy()) >=0.5
print("The accuracy for Logistic Regression Model is: ", 100* accuracy_score(tr,op))

The accuracy for Logistic Regression Model is:  85.64968344889961
